# Protocol for analysis of labeled proteomics data

This is the link to the github repository of the protocol 
Version: latest (still in development)
https://github.com/ProtProtocols/isolabeledprotocol
TODO add version!

Link to docker image:
_docker pull veitveit/isolabeledprotocol:latest_


## Abstract
Mass spectrometry with peptide labeling for quantification via reporter ions. Workflow based on open-source tools SearchGUI and PeptideShaker as well as in-house R scripts for further analysis using the isobar library for the extraction of reporter ions.


## Maintainer
Veit Schwämmle, veits@bmb.sdu.dk, github: veitveit
Johannes Griss, ...

## Software
Specify links for documentation and tutorials of used software, source code, publications and use cases. Detail versions of each used software. Alternatively, provide links to the software descriptions in https://bio.tools where this information is available.

- SearchGUI: https://bio.tools/searchgui, version 2.8.6
- Peptideshaker: https://bio.tools/peptideshaker, version 1.10.3
- isobar R library: https://bio.tools/isobar, version 1.24.0

## Diagram
Provide a simple diagram of functionality of the workflow/software. We recommend using controlled vocabularies for input/output data types and file formats as well as provided operation of the tool(s). You can use http://edamontology.org terms for the description.

__TODO: example__

## System requirements
Fill in the following items:
Required hard disk space for docker image, input and output files: Min. 3GB

Required memory: Min. 4GB

Recommmended number of threads: No requirement

## Example 
Presentation of well-documented instructions and commands to run the example use case. Depending on the use case and the software, provide link(s) to open the web service incorportated in the Docker image (e.g. 0.0.0.0:8080), bash commands to run programs from the command line and additional code for e.g. checking and visualizing the (intermediate) results. 

Instead of providing the instructions in this notebook, one can also provide a link to a notebook containing the example use case.

## More general use case (optional)
Provide link to notebook with a generalized use case that easily can be adapted to e.g. process different input data and concurrent parametrization.




# Example use case
The example takes the spectra from Test.mgf in the IN folder, searches them againts the human swissprot database and then carries out the quantification in R.

_Below a selection of parameters that can be modified directly:_


In [ ]:
%load_ext rpy2.ipython



In [ ]:
import ipywidgets as widgets
from ipywidgets import VBox, Label

w = widgets.IntSlider(min=-10,max=30,step=1,value=20)
w2 = widgets.BoundedFloatText(min=0,max=200,value=0.05)
w3 = widgets.Text("IN/sp_human.fasta")
# TODO  needs table to describe labeling formats
w4 = widgets.Dropdown(options={'TMT6': 'TMT 6-plex of K,TMT 6-plex of peptide N-term',
                               'TMT10': 'TMT 10-plex of K,TMT 10-plex of peptide N-term','iTRAQ4': 'iTRAQ 4-plex of K,iTRAQ 4-plex of Y,iTRAQ 4-plex of peptide N-term',
                               'iTRAQ8 (fixed)': 'iTRAQ 8-plex of K, iTRAQ 8-plex of peptide N-term',
                               'iTRAQ8 (variable)': 'iTRAQ 8-plex of Y'},value='TMT 10-plex of K,TMT 10-plex of peptide N-term')
w5= widgets.IntSlider(min=0,max=10,step=1,value=1)
w6 = widgets.Dropdown(options=["Carbamidomethylation of C","None"])
w7 = widgets.Dropdown(options=["None","Oxidation of M","Phosphorylation of STY"])
w8 = widgets.Text("IN/")

ww = widgets.Checkbox(description="Decoy")

display(VBox([Label('Precursor tolerance (ppm):'), w, 
              Label('Fragment ion tolerance (da):'),w2,
             Label('Fasta file (database):'),w3,
             Label('Quantification method:'),w4,
             Label('Number of miscleavages;'),w5,
             Label('Further fixed modifications'),w6,
             Label('Further variable modifications'),w7,
             Label('Folder for spectra files (files need to be mgf)'),w8]))

#TODO set names of samples and replicates (peptideshaker)


In [ ]:
%%bash -s "$w.value" "$w2.value" "$w3.value" "$w4.value" "$w5.value" "$w6.value" "$w7.value" "$w7.value"
function check_error {
    RETURN_CODE="$1"
    MSG="$2"

    if [ "${RETURN_CODE}" != "0" ]; then
        echo "Error: $MSG"
        exit 1
    fi
}
cd OUT
java -cp /home/biodocker/bin/SearchGUI-*/SearchGUI-*.jar eu.isas.searchgui.cmd.FastaCLI -in "../$3" -decoy

check_error $? "Failed to create decoy database"

DECOY_FASTA="../${3%.*}_concatenated_target_decoy.fasta"
echo $DECOY_FASTA

if [ ! -e ${DECOY_FASTA} ]; then
    echo "Failed to create decoy database"
    exit 1
fi

VAR_MODS=""
if [ $7 != "None" ]; then
    VAR_MODS="-variable_mods $7"
fi

# ---- create parameter file for SearchGUI ----
java -cp /home/biodocker/bin/SearchGUI-*/SearchGUI-*.jar \
eu.isas.searchgui.cmd.IdentificationParametersCLI -prec_tol $1 -frag_tol $2 \
-fixed_mods "$4,$6"  $VAR_MODS  -db "${DECOY_FASTA}" -out search.par -mc $5

check_error $? "Failed to create parameter file"

if [ ! -e "search.par" ]; then
    echo "Failed to create search parameters"
    exit 1
fi



In [ ]:
%%bash -s "$w8.value"
function check_error {
    RETURN_CODE="$1"
    MSG="$2"

    if [ "${RETURN_CODE}" != "0" ]; then
        echo "Error: $MSG"
        exit 1
    fi
}
FILE_LIST=$(ls $1)
echo $FILE_LIST
cd OUT

# Run Search
java -cp /home/biodocker/bin/SearchGUI-*/SearchGUI-*.jar eu.isas.searchgui.cmd.SearchCLI \
-spectrum_files ../$1  -output_folder ./  -id_params search.par -xtandem 0 -msgf 1 \
-comet 0 -ms_amanda 0 -myrimatch 0 -andromeda 0 -omssa 0 -tide 0
check_error $? "Search failed."


In [ ]:
#TODO, some visualizations + numbers (e.g. number of spectra, ...)

In [ ]:
%%bash -s "$w8.value"
function check_error {
    RETURN_CODE="$1"
    MSG="$2"

    if [ "${RETURN_CODE}" != "0" ]; then
        echo "Error: $MSG"
        exit 1
    fi
}

cd OUT
ls

# ---- PeptideShaker ----
java -Xmx4G  -cp /home/biodocker/bin/PeptideShaker-*/PeptideShaker-*.jar \
eu.isas.peptideshaker.cmd.PeptideShakerCLI -experiment experiment1 \
-sample test -replicate 1 -identification_files './'  -out ./experiment.cpsx \
-id_params search.par -spectrum_files  "../$1"

check_error $? "Failed to run PeptideShaker"

java -cp /home/biodocker/bin/PeptideShaker-*/PeptideShaker-*.jar \
eu.isas.peptideshaker.cmd.ReportCLI -in "experiment.cpsx" -out_reports "./" -reports "8"


In [ ]:
%%R 

# library causes the execution to fail if the library is missing
library(isobar)

# TODO load parameters "%%R -i parname"
# process the input files
max.fdr <- 0.01
quant.method <- "TMT10plexSpectra"
class.labels <- c("A", "B", "C", "D", "E", "F", "G", "H", "I", "J")
args <- commandArgs(trailingOnly = TRUE)

ident.file <- "OUT/experiment1_test_1_Extended_PSM_Report.txt"
mgf.files <- system("ls IN/*.mgf", intern=T)

if (!file.exists(ident.file)) {
    stop("Error: Cannot find identification file ", ident.file)
}
for (mgf.file in mgf.files) {
    if (!file.exists(mgf.file)) {
        stop("Error: Cannot find MGF file ", mgf.file)
    }
}

# Convert SearchGUI output to isobar format
psms <- read.csv(ident.file, sep = "\t")

if (! "Decoy" %in% names(psms)) {
    stop("Error: No decoy information available in output file")
}

print(paste("Loaded",nrow(psms), "PSMs"))

# ---- Confidence filter ----
psms <- psms[order(psms[, "Confidence...."], decreasing = T), ]
decoy.psms <- which(psms[, "Decoy"] == "1")

decoy.count <- 0

for (decoy.index in decoy.psms) {
    decoy.count <- decoy.count + 1
    target.count <- decoy.index - decoy.count

    cur.fdr <- (decoy.count * 2) / (decoy.count + target.count)

    if (cur.fdr > max.fdr) {
        # filter
        psms <- psms[1:decoy.index - 1,]
        break
    }
}

# print(head(psms))

print(paste0("Filtered ", nrow(psms), " PSMs @ ", max.fdr, " FDR"))

# ---- convert to isobar output ----
cols.to.save <- c("Protein.s.", "Sequence", "Spectrum.Title", "Variable.Modifications", "Confidence....", "D.score", "Validation", "Precursor.m.z.Error..ppm.", "Spectrum.File")

if (!all(cols.to.save %in% colnames(psms))) {
    stop("Error: Unexpected result format")
}

psms <- psms[, cols.to.save]
colnames(psms) <- c("accession", "peptide", "spectrum", "var_mod", "pepscore", "dscore", "validation", 
"precursor.mz.error.ppm", "file")

# TODO: add modif...
psms$modif <- ""

write.table(psms, file = "t.corr.csv", sep = "\t", row.names = F, quote = F)

# ---- isobar workflow ----
ib <- readIBSpectra(quant.method, "t.corr.csv", mgf.files, decode.titles = T)

ib <- correctIsotopeImpurities(ib)
ib <- normalize(ib)

# Extract peptide and protein ratios
protein.ratios <- proteinRatios(ib, noise.model, combn.method = "versus.channel", cl = class.labels)
peptide.ratios <- peptideRatios(ib, noise.model, combn.method = "versus.channel", cl = class.labels)
noise.model <- NoiseModel(ib)

boxplot(peptide.ratios$lratio, main = "Peptide ratios", ylab = "Peptide ratio")
boxplot(protein.ratios$lratio, main = "Protein ratios", ylab = "Protein reatios")

# save the ratios for later use
saveRDS(protein.ratios, file = "OUT/protein.ratios.rds")
saveRDS(peptide.ratios, file = "OUT/peptide.ratios.rds")